In [33]:
import pandas as pd
import pickle
import pyarrow
import fastparquet
model = None

In [3]:
data_file = 'troop_movements10m.csv'
data = pd.read_csv(data_file)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      object 
 1   unit_id        int64  
 2   unit_type      object 
 3   location_x     float64
 4   location_y     float64
 5   destination_x  int64  
 6   destination_y  int64  
 7   homeworld      object 
dtypes: float64(2), int64(3), object(3)
memory usage: 610.4+ MB


In [8]:
data = data.replace(to_replace='invalid_unit', value='unknown')
data.tail()

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld
9999995,2023-02-18 10:26:20,9999996,resistance_soldier,6.0,6.0,1,3,Troiken
9999996,2023-02-18 10:26:19,9999997,tie_silencer,8.0,6.0,2,0,Kashyyyk
9999997,2023-02-18 10:26:18,9999998,tie_fighter,7.0,7.0,6,4,Kashyyyk
9999998,2023-02-18 10:26:17,9999999,tie_fighter,6.0,6.0,8,8,Kalee
9999999,2023-02-18 10:26:16,10000000,unknown,NaN,NaN,3,0,Skako


In [10]:
data = data.ffill()
data.tail()

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld
9999995,2023-02-18 10:26:20,9999996,resistance_soldier,6.0,6.0,1,3,Troiken
9999996,2023-02-18 10:26:19,9999997,tie_silencer,8.0,6.0,2,0,Kashyyyk
9999997,2023-02-18 10:26:18,9999998,tie_fighter,7.0,7.0,6,4,Kashyyyk
9999998,2023-02-18 10:26:17,9999999,tie_fighter,6.0,6.0,8,8,Kalee
9999999,2023-02-18 10:26:16,10000000,unknown,6.0,6.0,3,0,Skako


In [11]:
file_name = 'troop_movements10m.parquet' 
data.to_parquet(file_name)

In [34]:
pickle_file = 'trained_model.pkl' 
with open (pickle_file, 'rb') as file:
    model = pickle.load(file)

In [35]:
importances = model.feature_importances_
print(importances)

[0.0122353  0.0199622  0.00621437 0.00900902 0.00276397 0.00505955
 0.00420137 0.01431982 0.01898708 0.0182327  0.00948232 0.01666303
 0.01068442 0.02018704 0.01389231 0.01239215 0.02178665 0.00519954
 0.01573345 0.01630502 0.00842646 0.01854667 0.00973288 0.00549032
 0.00946497 0.0201837  0.01276509 0.01633555 0.01130719 0.01006759
 0.0164171  0.00870728 0.01275465 0.01232987 0.00963276 0.03271729
 0.01565614 0.00600962 0.00805594 0.02193916 0.00799224 0.03804445
 0.04947512 0.01196476 0.03252116 0.06233393 0.04082234 0.08576673
 0.09411171 0.05711602]


In [15]:
parquet_file = 'troop_movements10m.parquet' 
cleaned_data = pd.read_parquet(parquet_file)

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld
0,2023-06-13 17:33:18,1,at-st,2.0,8.0,1,1,Glee Anselm
1,2023-06-13 17:33:17,2,tie_silencer,4.0,4.0,0,1,Trandosha
2,2023-06-13 17:33:16,3,at-at,0.0,3.0,6,1,Corellia
3,2023-06-13 17:33:15,4,tie_silencer,6.0,1.0,6,9,Shili
4,2023-06-13 17:33:14,5,tie_fighter,0.0,4.0,9,6,Muunilinst


In [38]:
df = pd.DataFrame(cleaned_data, columns=['homeworld', 'unit_type'])
df_encoded = pd.get_dummies(df)
predictions = model.predict(df_encoded)


In [41]:
cleaned_data['predictions'] = predictions
cleaned_data.head()

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld,predictions
0,2023-06-13 17:33:18,1,at-st,2.0,8.0,1,1,Glee Anselm,empire
1,2023-06-13 17:33:17,2,tie_silencer,4.0,4.0,0,1,Trandosha,empire
2,2023-06-13 17:33:16,3,at-at,0.0,3.0,6,1,Corellia,resistance
3,2023-06-13 17:33:15,4,tie_silencer,6.0,1.0,6,9,Shili,empire
4,2023-06-13 17:33:14,5,tie_fighter,0.0,4.0,9,6,Muunilinst,empire
